## 数据输入（一） numpy 数据
之前都是再用 tfrecord 打包的方式来做的，tf1.3 以后提供了 tf.data API，现在来学习一下。

这个主要参考了[TensorFlow全新的数据读取方式：Dataset API入门教程](https://zhuanlan.zhihu.com/p/30751039)

In [1]:
import warnings
warnings.filterwarnings('ignore')  # 不打印 warning 

import tensorflow as tf

# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

import numpy as np

## 1.打包 tf.data.Dataset

### 2.1 最简单的例子

In [2]:
X = np.arange(20).reshape([10, 2])
y = np.arange(10).reshape([10,1])
print(y[:5])

[[0]
 [1]
 [2]
 [3]
 [4]]


[文档](https://www.tensorflow.org/programmers_guide/datasets)中说：

>要启动输入管道，您必须定义来源。例如，要通过内存中的某些张量构建 Dataset，您可以使用 tf.data.Dataset.from_tensors() 或 tf.data.Dataset.from_tensor_slices()。或者，如果您的输入数据以推荐的 TFRecord 格式存储在磁盘上，那么您可以构建 tf.data.TFRecordDataset

现在我们先来看看 from_tensor_slices() 的用法。

In [3]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))
print(dataset)

<TensorSliceDataset shapes: ((2,), (1,)), types: (tf.int64, tf.int64)>


这样我们就把 X,y 构建成了一个 dataset，可以看到 X 和 y 的维度，但是并不能看到样本的数量。

下一步，我们要取出 dataset 中的数据，需要创建一个迭代器。tf 提供了好多种创建迭代器的方式，我发现目前来说使用最简单的方法就基本能够满足要求了。

In [4]:
iterator = dataset.make_one_shot_iterator()
print(iterator)

In [5]:
import sys

sess.run(tf.global_variables_initializer())

count = 0
try:
    while True:
        x_batch, y_batch = sess.run(iterator.get_next())
        print('count = {} : y = {}'.format(count, y_batch))
        count += 1
except Exception as e:
    print('\n', e)
    print('final count = {}'.format(count))
        

count = 0 : y = [0]
count = 1 : y = [1]
count = 2 : y = [2]
count = 3 : y = [3]
count = 4 : y = [4]
count = 5 : y = [5]
count = 6 : y = [6]
count = 7 : y = [7]
count = 8 : y = [8]
count = 9 : y = [9]

 End of sequence
	 [[Node: IteratorGetNext_10 = IteratorGetNext[output_shapes=[[2], [1]], output_types=[DT_INT64, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator)]]

Caused by op 'IteratorGetNext_10', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  

从上面的结果可以看到，我们使用 `iterator = dataset.make_one_shot_iterator()` 的方式创建的迭代器可以每次取出一个样本， one_shot 表示只能迭代一次，迭代完了以后再取值就会报错了。

### 2.2 相关功能

在实际应用中，tf 提供了一下几个函数来实现我们的要求：下面摘自 https://zhuanlan.zhihu.com/p/30751039


**map**

map接收一个函数，Dataset中的每个元素都会被当作这个函数的输入，并将函数返回值作为新的Dataset，如我们可以对dataset中每个元素的值加1：
```
dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))
dataset = dataset.map(lambda x: x + 1) # 2.0, 3.0, 4.0, 5.0, 6.0
```

**batch**

batch就是将多个元素组合成batch，如下面的程序将dataset中的每个元素组成了大小为32的batch：
```
dataset = dataset.batch(32)
```

**shuffle**

shuffle的功能为打乱dataset中的元素，它有一个参数buffersize，表示打乱时使用的buffer的大小：
```
dataset = dataset.shuffle(buffer_size=10000)
```

**repeat**

repeat的功能就是将整个序列重复多次，主要用来处理机器学习中的epoch，假设原先的数据是一个epoch，使用repeat(5)就可以将之变成5个epoch：
```
dataset = dataset.repeat(5)
```
如果**直接调用repeat()的话，生成的序列就会无限重复下去，没有结束**，因此也不会抛出tf.errors.OutOfRangeError异常：
```
dataset = dataset.repeat()
```


In [1]:
import warnings
warnings.filterwarnings('ignore')  # 不打印 warning 

import tensorflow as tf

# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

import numpy as np
import sys


X = np.arange(20).reshape([10, 2])
y = np.arange(10).reshape([10,1])
print(X[:5])
print(y[:5])

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
[[0]
 [1]
 [2]
 [3]
 [4]]


下面我们要实现每次取出 4 个值，并且把 x 中每个元素×2

In [2]:
def pre_func(X, y):
    X = tf.multiply(X, 2)
    return X, y

batch_size = 6
# 创建 dataset
dataset = tf.data.Dataset.from_tensor_slices((X, y))

# 对数据进行操作
dataset = dataset.map(pre_func)
dataset = dataset.repeat(3).batch(4)
print(dataset)

# 生成迭代器
iterator = dataset.make_one_shot_iterator()
print(iterator)


sess.run(tf.global_variables_initializer())

# 迭代取值
count = 0
try:
    while True:
        X_batch, y_batch = sess.run(iterator.get_next())
        print('count = {} : X = {}'.format(count, X_batch))
        count += 1
except Exception as e:
    print('final count = {}'.format(count))
  

<BatchDataset shapes: ((?, 2), (?, 1)), types: (tf.int64, tf.int64)>


下面添加 shuffle 功能

In [7]:
def pre_func(X, y):
    X = tf.multiply(X, 2)
    return X, y

batch_size = 6
# 创建 dataset
dataset = tf.data.Dataset.from_tensor_slices((X, y))

# 对数据进行操作
dataset = dataset.map(pre_func)
dataset = dataset.shuffle(buffer_size=20)  # 设置 buffle_size 越大，shuffle 越均匀
dataset = dataset.repeat().batch(4)
print(dataset)

# 生成迭代器
iterator = dataset.make_one_shot_iterator()
print(iterator)


sess.run(tf.global_variables_initializer())

# 迭代取值
for count in range(10):
    X_batch, y_batch = sess.run(iterator.get_next())
    print('count = {} : y = {}'.format(count, y_batch.reshape([-1])))

<BatchDataset shapes: ((?, 2), (?, 1)), types: (tf.int64, tf.int64)>
count = 0 : y = [1 9 6 7]
count = 1 : y = [3 5 0 8]
count = 2 : y = [2 4 3 6]
count = 3 : y = [7 2 1 0]
count = 4 : y = [5 8 9 4]
count = 5 : y = [7 6 2 9]
count = 6 : y = [0 3 8 1]
count = 7 : y = [4 5 2 0]
count = 8 : y = [6 9 4 5]
count = 9 : y = [8 3 7 1]


从上面的结果来看，对于能够一次性读入内存中的数据。dataset 具备了非常好的功能：
- 使用 map 函数可以对数据进行操作
- 使用 shuffle 能够很好地扰乱数据，如果 先 shuffle 再 repeat 的话，可以每次都无重复地迭代完一个 epoch 再到下一个epoch；如果是先 repeat 再 shuffle 的话可能每个batch中会出现相同的样本。
- 使用 repeat 函数可以不断地取样本
- 使用 batch 函数可以很方便的读取数据

## 2.3 使用 placeholder() 读取大数组

刚才的方式读取数据非常方便，但是存在一个问题就是在构建 dataset 的时候：
```python
dataset = tf.data.Dataset.from_tensor_slices((X, y))
```
他会把整个 dataset 当做图的一部分，也就是说，X, y 如果很大的话，那么图就会很大，这样在处理大数组的时候并不好。

对于较大数组的读取，我们需要使用： `dataset.make_initializable_iterator()` 和 `tf.placeholder` 来解决。


In [1]:
import warnings
warnings.filterwarnings('ignore')  # 不打印 warning 

import tensorflow as tf

# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

import numpy as np
import sys


X = np.arange(1000).reshape([500, 2])
y = np.arange(500).reshape([500,1])
print(X.shape, y.shape)

(500, 2) (500, 1)


In [2]:
# 使用 placeholder 来取代 array，并使用 initiable iterator， 在需要的时候再将 array 传进去
# 这样能够避免把大数组保存在图中
X_placeholder = tf.placeholder(dtype=X.dtype, shape=X.shape)
y_placeholder = tf.placeholder(dtype=y.dtype, shape=y.shape)

batch_size = 6
# 创建 dataset
dataset = tf.data.Dataset.from_tensor_slices((X_placeholder, y_placeholder))
print('dataset is', dataset)

# 对数据进行操作
def pre_func(X, y):
    X = tf.multiply(X, 2)
    return X, y

dataset = dataset.map(pre_func)
# dataset = dataset.shuffle(buffer_size=20)  # 设置 buffle_size 越大，shuffle 越均匀
dataset = dataset.repeat().batch(4)
print('after get batch', dataset)

# 生成迭代器
iterator = dataset.make_initializable_iterator()
print(iterator)
sess.run(iterator.initializer, feed_dict={X_placeholder: X, y_placeholder:y})


# 迭代取值
count = 0
for count in range(10):
    X_batch, y_batch = sess.run(iterator.get_next())
    print('count = {} : y = {}'.format(count, y_batch.reshape([-1])))

dataset is <TensorSliceDataset shapes: ((2,), (1,)), types: (tf.int64, tf.int64)>
after get batch <BatchDataset shapes: ((?, 2), (?, 1)), types: (tf.int64, tf.int64)>
count = 0 : y = [0 1 2 3]
count = 1 : y = [4 5 6 7]
count = 2 : y = [ 8  9 10 11]
count = 3 : y = [12 13 14 15]
count = 4 : y = [16 17 18 19]
count = 5 : y = [20 21 22 23]
count = 6 : y = [24 25 26 27]
count = 7 : y = [28 29 30 31]
count = 8 : y = [32 33 34 35]
count = 9 : y = [36 37 38 39]


其实这样写起来比前面的 `dataset.make_one_shot_iterator()` 是麻烦了一点点，但是也没有太麻烦。

下面是对 MNIST 数据训练集 55000 个样本 读取的一个速度比较，统一 `batch_size=128`，主要比较 `one-shot` 和 `initializable` 两种迭代方式：

|iter_mode|buffer_size|100 batch(s)|
|:----:|:---:|:---:|
|one-shot|2000|125|
|one-shot|5000|149|
|initializable|2000|0.7|
|initializable|5000|0.7|

可以看到，使用 `initializable` 方式的速度明显要快很多。因为使用 `one-shot` 方式会把整个矩阵放在图中，计算非常非常慢。